In [1]:
import os
import cv2
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [6]:
def resize_annotation(annotation_path, new_width, new_height, output_dir):
    tree = ET.parse(annotation_path)
    root = tree.getroot()

    size = root.find('size')
    width = int(size.find('width').text)
    height = int(size.find('height').text)

    size.find('width').text = str(new_width)
    size.find('height').text = str(new_height)

    for obj in root.findall('object'):
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        xmin = int(xmin * (new_width / width))
        ymin = int(ymin * (new_height / height))
        xmax = int(xmax * (new_width / width))
        ymax = int(ymax * (new_height / height))

        bndbox.find('xmin').text = str(xmin)
        bndbox.find('ymin').text = str(ymin)
        bndbox.find('xmax').text = str(xmax)
        bndbox.find('ymax').text = str(ymax)

    output_annotation_name = os.path.basename(annotation_path)
    output_annotation_path = os.path.join(output_dir, output_annotation_name)
    tree.write(output_annotation_path)

In [10]:
new_width = 1050
new_height = 1585

image_dir = '../../Загрузки/project/images'
annotation_dir = '../../Загрузки/project/Annotations'
output_dir = '../../Загрузки/project/Resized'

for filename in tqdm(os.listdir(image_dir)):
    if not filename.endswith('.png'):
        continue

    image_path = os.path.join(image_dir, filename)
    image = cv2.imread(image_path)

    resized_image = cv2.resize(image, (new_width, new_height))
    resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

    output_image_path = os.path.join(output_dir, filename)
    cv2.imwrite(output_image_path, resized_image)

    annotation_filename = filename.replace('.png', '.xml')
    annotation_path = os.path.join(annotation_dir, annotation_filename)

    
    resize_annotation(annotation_path, new_width, new_height, output_dir)

100%|█████████████████████████████████████████| 156/156 [00:06<00:00, 22.73it/s]
